# Trabalho prático: Implementação de uma Rede Neural Convolucional
## Objetivo:
Implementar e treinar um modelo de Rede Neural Convolucional para classificação de imagens de cães e de gatos e
entregar um relatório técnico, desenvolvido por meio da plataforma Jupyter Notebook.

## Fundamentação téorica:
Uma Rede Neural Convolucional (ou Convolutional Neural Network - CNN) e uma variacão das redes de Perceptrons de Multiplas Camadas, tendo sido inspirada no processo biologico de processamentos de dados visuais. De maneira semelhante aos processos tradicionais de visao computacional, uma CNN e capaz de aplicar filtros em dados visuais, mantendo a relação de vizinhançã entre os pixels da imagem ao longo do processamento da rede. Uma rede neural convolucional consiste em multiplas partes com funções diferentes. Inicialmente é comum aplicar sobre o dado de entrada camadas ditas de convolução. Uma camada de convoluao é composta por diversos neuronios, cada um responsavel por aplicar um filtro em um pedação específico da imagem. Podemos imaginar cada neuronio sendo  onectado a um conjunto de pixels da camada anterior e que a cada uma dessa conexoes se atribui um peso. A combinação das entradas de um neuronio, utilizando os pesos respectivos de cada uma de suas conexoes, produz uma saıda passada para a camada seguinte. Os pesos atribuıdos as conexões de um neuronio podem ser interpretados como uma matriz que representa o filtro de uma convolução de imagens no domınio espacial.

## Metodologia
Para implementação da rede neural foi utilizado o Keras que é uma biblioteca de rede neural de código aberto escrita em Python. Ele é capaz de rodar em cima de TensorFlow, Microsoft Cognitive Toolkit, Theano, ou PlaidML.
Projetado para permitir a rápida experimentação de redes neurais profundas, ele se concentra em ser fácil de usar, modular e extensível.

### 1. Importar o Keras e todas as bibliotecas necessarias


In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import model_from_json

Using TensorFlow backend.


### 2. Definir a arquitetura da CNN

In [2]:
def build():
    cnn = Sequential()
    cnn.add(Conv2D(64, kernel_size=3, input_shape = (64, 64, 3), activation = 'relu'))
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    
    cnn.add(Conv2D(32, kernel_size=3, activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    
    cnn.add(Flatten())
    
    cnn.add(Dense(units=128, activation='relu'))
    cnn.add(Dense(units=1, activation='sigmoid'))
    
    cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return cnn

 Esta CNN é baseada na arquitetura LeNet-5 
 <img src="dataset/lenet5.png" width="77%" title="MCP Neuron">
Resumidamente, a arquitetura da LeNet-5 é composta por uma sequência com as seguintes camadas:
- CNN é composta por um conjunto de 6 filtros (5×5), stride=1.
- POOL (2×2), stride=2, para reduzir o tamanho espacial das matrizes resultantes.
- CNN (5×5) com 16 filtros e stride=1.
- POOL (2×2), stride=2.
- Os mapas de características são achatados (flatten), formando 400 nós (5x5x16) para a próxima camanda FC.
- FC com 120 nós.
- FC com 84 nós.

A CNN é instanciada pela classe Sequential, e cada layer é adicionado na sequência do outro, seguindo a arquitetura já detalhada acima.

### 3. Carregar os dados para treinar e testar
Para treinar e testar a CNN foram utilizados 8000 fotos para treino e 2000 para teste de caes e gatos

In [3]:
def dataset():
    trainer_data = ImageDataGenerator(rescale=1./255,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True)
    
    test_data = ImageDataGenerator(rescale=1./255)
    
    trainer_set = trainer_data.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

    test_set = test_data.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')
    
    return trainer_set, test_set

### 4. Treinado a CNN
Classe LeNet5 implementada, dados de entrada tratados corretamente, e agora é hora de compilar a CNN e treiná-la 

In [5]:
classifier = build()
    
trainer, test = dataset()
    
classifier.fit_generator(trainer,
                        steps_per_epoch=8000,
                        epochs=3,
                        validation_data=test,
                        validation_steps=2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/3
8000/8000 [==============================] - 1663s 208ms/step - loss: 0.3609 - acc: 0.8306 - val_loss: 0.5949 - val_acc: 0.8103
Epoch 2/3
8000/8000 [==============================] - 1826s 228ms/step - loss: 0.1104 - acc: 0.9578 - val_loss: 0.9716 - val_acc: 0.7813
Epoch 3/3
8000/8000 [==============================] - 1680s 210ms/step - loss: 0.0591 - acc: 0.9788 - val_loss: 1.0464 - val_acc: 0.8049


### 5. Avaliando a CNN
Para avaliar o desempenho da nossa CNN,  chamamos o método model.predict para gerar previsões em cima do dataset de teste.

In [14]:
import numpy as np

test = image.load_img('dataset/cat_or_dog.jpg', target_size = (64,64))
test = image.img_to_array(test)
test = np.expand_dims(test, axis=0)

res = classifier.predict(test)

print("dog") if res[0][0] == 1 else print("cat")

dog


<img src="dataset/cat_or_dog.jpg" width="22%" title="???" align="left">

## Análise dos Resultados
Conseguimos uma precisão de 97% nas previsões em 3 epocas feitas com a LeNet-5 treinada como o treinamento foi feito em um notebook com placa de video fraca o treinamento levo 1 hora 43 minutos para concluir  

## Conclusão
Atualmente inumeras abordagens utilizando Redes Neurais com aprendizado profundo estao alcançando os melhores resultados em diversos problemas de Visao Computacional. Por isso e importante entender como elas funcionam, 
para assim projetar melhores arquiteturas e obter melhores resultados.